In [3]:
%cd ..

/home/dongkyu/sDG


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, RandomSampler
from torchvision import models
from torchvision.datasets import CIFAR10
from torchvision.utils import make_grid
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

import os
import click
import time
import numpy as np
import copy

from loss_functions import SupConLoss, MdarLoss, kl_divergence
from network import mnist_net, res_net, cifar_net, generator
from network.modules import get_resnet, get_generator, freeze, unfreeze, freeze_, unfreeze_, LARS
from tools.miro_utils import *
from tools.farmer import *
import data_loader
from main_base import evaluate

import matplotlib.pyplot as plt


HOME = os.environ['HOME']

In [9]:
w_cls=1.0
w_cyc=20
w_info=0.1 
w_oracle=1.0
w_div=2.0 
div_thresh=0.5
w_tgt=1.0


#how about try smaller tgt 
n_tgt=100 #20
max_tgt=99 #19
tgt_epochs_fixg=5 #15
tgt_epochs=10 #30
lmda=0.051 #lmda for adv-barlowtwins (0.051 best)

gen='cnn'
interpolation='img'
oracle=False


data='pacs' #mnist/cifar10/pacs
backbone='resnet18' #(pacs_net/resnet18/resnet50/wideresnet) 
pretrained='True' #Only to load right base model. my_iter process is set as pretrained=False.
projection_dim=512 #default: (mnist: 128/ cifar-10:128)
loss_fn='mdar' #supcon/mdar
lr=1e-4 #1e-4 #1e-3 sucks #adam with 1e-5/1e-4 #sdg with 1e-4
lr_scheduler='none' #cosine/none  #adam with none # sgd with cosine
optimizer='adam' #sgd/adam

batchsize=16 #default:16 for pacs


In [11]:
ckpt='saved-model/pacs/base_resnet18_True_512_run0/best.pkl'

In [13]:
encoder = get_resnet(backbone, pretrained) # Pretrained Backbone default as False - We will load our model anyway
n_features = encoder.fc.in_features
output_dim = 7 
src_net= res_net.ConvNet(encoder, projection_dim, n_features, output_dim).cuda() #projection_dim/ n_features/output_dim=10
saved_weight = torch.load(ckpt)
src_net.load_state_dict(saved_weight['cls_net'])
if optimizer == 'adam':
    src_opt = optim.Adam(src_net.parameters(), lr=lr)
elif optimizer == 'sgd':
    src_opt = optim.SGD(src_net.parameters(), lr=lr, momentum=0.9, nesterov=True, weight_decay=1e-1)

In [26]:
src_net.encoder

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [30]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

In [31]:
src_net.encoder.layer1[1].bn2.register_forward_hook(get_features('feats'))

In [32]:
src_net.encoder.layer2[1].bn2.register_forward_hook(get_features('feats'))

In [33]:
src_net.encoder.layer3[1].bn2.register_forward_hook(get_features('feats'))

In [34]:
src_net.encoder.layer4[1].bn2.register_forward_hook(get_features('feats'))

In [ ]:
src_net.encoder.layer1[1].bn2.register_forward_hook(get_features('feats'))
src_net.encoder.layer2[1].bn2.register_forward_hook(get_features('feats'))
src_net.encoder.layer3[1].bn2.register_forward_hook(get_features('feats'))
src_net.encoder.layer4[1].bn2.register_forward_hook(get_features('feats'))

In [35]:
from tools.midlevel import IntermediateLayerGetter

In [36]:
return_layers= {"encoder.layer1[1].bn2"}

In [37]:
IntermediateLayerGetter(src_net, return_layers= return_layers, keep_output=True)